In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import csv
import random
import itertools
import sys
import shutil
from tqdm import tqdm
import helpermethods
from edgeml_tf.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml_tf.graph.protoNN import ProtoNN
import edgeml_tf.utils as utils
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from scipy.stats import uniform
from data_utils import *
import re
import time
import pickle
import glob
from tensorflow import keras
import tensorflow.compat.v1.keras.backend as K
from edgeml_tf.tflite.protoNNLayer import ProtoNNLayer

## Import Dataset

In [2]:
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'
model_dir = 'trained_models/'
window_size = 550
stride = 50
channels = 2

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=2)
print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

100%|█████████████████████████████████████████| 399/399 [00:05<00:00, 74.52it/s]


(14671, 550, 2)
(14671, 9)
(300, 550, 2)
(300, 9)


## Feature Extraction

In [3]:
feat_size = 10
featX_tr = np.zeros((X_tr.shape[0],feat_size))
featX_test = np.zeros((X_test.shape[0],feat_size))
for i in range(X_tr.shape[0]):
    cur_win = X_tr[i]
    featX_tr[i,0] = np.min(cur_win[:,0])
    featX_tr[i,1] = np.min(cur_win[:,1])
    featX_tr[i,2] = np.max(cur_win[:,0])
    featX_tr[i,3] = np.max(cur_win[:,1])
    featX_tr[i,4] = featX_tr[i,2]-featX_tr[i,0]
    featX_tr[i,5] = featX_tr[i,3]-featX_tr[i,1]
    featX_tr[i,6] = np.var(cur_win[:,0])
    featX_tr[i,7] = np.var(cur_win[:,1])
    featX_tr[i,8] = np.sqrt(featX_tr[i,6])
    featX_tr[i,9] = np.sqrt(featX_tr[i,7])  
    
for i in range(X_test.shape[0]):
    cur_win = X_test[i]
    featX_test[i,0] = np.min(cur_win[:,0])
    featX_test[i,1] = np.min(cur_win[:,1])
    featX_test[i,2] = np.max(cur_win[:,0])
    featX_test[i,3] = np.max(cur_win[:,1])
    featX_test[i,4] = featX_test[i,2]-featX_test[i,0]
    featX_test[i,5] = featX_test[i,3]-featX_test[i,1]
    featX_test[i,6] = np.var(cur_win[:,0])
    featX_test[i,7] = np.var(cur_win[:,1])
    featX_test[i,8] = np.sqrt(featX_test[i,6])
    featX_test[i,9] = np.sqrt(featX_test[i,7])
    
x_train = featX_tr
y_train = Y_tr
x_test = featX_test
y_test = Y_test
numClasses = Y_tr.shape[1]
dataDimension = x_train.shape[1]

mean = np.mean(x_train, 0)
std = np.std(x_train, 0)
std[std[:] < 0.000001] = 1
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std
Y_tr_int = np.argmax(Y_tr, axis=1)
Y_test_int = np.argmax(Y_test,axis=1)

shutil.rmtree('earable_dataset/', ignore_errors=True)
os.mkdir('earable_dataset/')
train = np.concatenate([Y_tr_int.reshape((Y_tr_int.shape[0],1)),x_train],axis=1)
with open('earable_dataset/train.npy','wb') as f:
    np.save(f,train)
test = np.concatenate([Y_test_int.reshape((Y_test_int.shape[0],1)),x_test],axis=1)
with open('earable_dataset/test.npy','wb') as f:
    np.save(f,test)

# Training and Hyperparameter Tuning

In [4]:
NUM_EPOCHS = 1000 #epochs to train each model for

In [5]:
PROJECTION_DIM = 70
NUM_PROTOTYPES = 70
GAMMA = 0.004

REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.03
BATCH_SIZE =32    

W, B, gamma = helpermethods.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                   NUM_PROTOTYPES, x_train)


X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()
trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600, valStep=1) 
shutil.rmtree(model_dir, ignore_errors=True)
os.mkdir(model_dir)
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
W, B, Z, gamma = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
gamma = sess.run(gamma)
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helpermethods.getModelSize(matrixList, sparcityList, expected=False)
np.save(model_dir + '/W.npy', matrixList[0])
np.save(model_dir + '/B.npy', matrixList[1])
np.save(model_dir + '/Z.npy', matrixList[2])
np.save(model_dir + '/gamma.npy', gamma)

Epoch:   0 Batch:   0 Loss: 8.03941 Accuracy: 0.12500
Test Loss: 1.03865 Accuracy: 0.67000
Epoch:   1 Batch:   0 Loss: 1.01930 Accuracy: 0.62500
Test Loss: 0.86270 Accuracy: 0.67000
Epoch:   2 Batch:   0 Loss: 0.84018 Accuracy: 0.65625
Test Loss: 0.81319 Accuracy: 0.68333
Epoch:   3 Batch:   0 Loss: 0.77531 Accuracy: 0.65625
Test Loss: 0.78936 Accuracy: 0.69333
Epoch:   4 Batch:   0 Loss: 0.73715 Accuracy: 0.65625
Test Loss: 0.77453 Accuracy: 0.68667
Epoch:   5 Batch:   0 Loss: 0.71665 Accuracy: 0.65625
Test Loss: 0.76459 Accuracy: 0.69000
Epoch:   6 Batch:   0 Loss: 0.70481 Accuracy: 0.65625
Test Loss: 0.75658 Accuracy: 0.69667
Epoch:   7 Batch:   0 Loss: 0.69725 Accuracy: 0.65625
Test Loss: 0.74987 Accuracy: 0.70667
Epoch:   8 Batch:   0 Loss: 0.69184 Accuracy: 0.65625
Test Loss: 0.74440 Accuracy: 0.70667
Epoch:   9 Batch:   0 Loss: 0.68908 Accuracy: 0.65625
Test Loss: 0.73930 Accuracy: 0.72333
Epoch:  10 Batch:   0 Loss: 0.68632 Accuracy: 0.68750
Test Loss: 0.73533 Accuracy: 0.72667

Test Loss: 0.68008 Accuracy: 0.74000
Epoch:  91 Batch:   0 Loss: 0.62519 Accuracy: 0.71875
Test Loss: 0.68017 Accuracy: 0.74000
Epoch:  92 Batch:   0 Loss: 0.62498 Accuracy: 0.71875
Test Loss: 0.68030 Accuracy: 0.74000
Epoch:  93 Batch:   0 Loss: 0.62477 Accuracy: 0.71875
Test Loss: 0.68041 Accuracy: 0.73667
Epoch:  94 Batch:   0 Loss: 0.62459 Accuracy: 0.71875
Test Loss: 0.68053 Accuracy: 0.73667
Epoch:  95 Batch:   0 Loss: 0.62437 Accuracy: 0.75000
Test Loss: 0.68072 Accuracy: 0.74000
Epoch:  96 Batch:   0 Loss: 0.62434 Accuracy: 0.75000
Test Loss: 0.68089 Accuracy: 0.74333
Epoch:  97 Batch:   0 Loss: 0.62420 Accuracy: 0.75000
Test Loss: 0.68110 Accuracy: 0.74333
Epoch:  98 Batch:   0 Loss: 0.62410 Accuracy: 0.75000
Test Loss: 0.68133 Accuracy: 0.74667
Epoch:  99 Batch:   0 Loss: 0.62401 Accuracy: 0.75000
Test Loss: 0.68153 Accuracy: 0.74333
Epoch: 100 Batch:   0 Loss: 0.62395 Accuracy: 0.75000
Test Loss: 0.68174 Accuracy: 0.74333
Epoch: 101 Batch:   0 Loss: 0.62387 Accuracy: 0.75000

Test Loss: 0.69503 Accuracy: 0.71333
Epoch: 182 Batch:   0 Loss: 0.63079 Accuracy: 0.68750
Test Loss: 0.69507 Accuracy: 0.71333
Epoch: 183 Batch:   0 Loss: 0.63078 Accuracy: 0.68750
Test Loss: 0.69512 Accuracy: 0.71333
Epoch: 184 Batch:   0 Loss: 0.63082 Accuracy: 0.68750
Test Loss: 0.69516 Accuracy: 0.71333
Epoch: 185 Batch:   0 Loss: 0.63084 Accuracy: 0.68750
Test Loss: 0.69521 Accuracy: 0.71333
Epoch: 186 Batch:   0 Loss: 0.63086 Accuracy: 0.68750
Test Loss: 0.69533 Accuracy: 0.71333
Epoch: 187 Batch:   0 Loss: 0.63088 Accuracy: 0.68750
Test Loss: 0.69534 Accuracy: 0.71333
Epoch: 188 Batch:   0 Loss: 0.63092 Accuracy: 0.68750
Test Loss: 0.69539 Accuracy: 0.71333
Epoch: 189 Batch:   0 Loss: 0.63094 Accuracy: 0.68750
Test Loss: 0.69543 Accuracy: 0.71333
Epoch: 190 Batch:   0 Loss: 0.63096 Accuracy: 0.68750
Test Loss: 0.69546 Accuracy: 0.71333
Epoch: 191 Batch:   0 Loss: 0.63099 Accuracy: 0.68750
Test Loss: 0.69549 Accuracy: 0.71333
Epoch: 192 Batch:   0 Loss: 0.63098 Accuracy: 0.68750

Test Loss: 0.69673 Accuracy: 0.71333
Epoch: 273 Batch:   0 Loss: 0.63043 Accuracy: 0.68750
Test Loss: 0.69671 Accuracy: 0.71333
Epoch: 274 Batch:   0 Loss: 0.63042 Accuracy: 0.68750
Test Loss: 0.69671 Accuracy: 0.71333
Epoch: 275 Batch:   0 Loss: 0.63040 Accuracy: 0.68750
Test Loss: 0.69671 Accuracy: 0.71333
Epoch: 276 Batch:   0 Loss: 0.63039 Accuracy: 0.68750
Test Loss: 0.69671 Accuracy: 0.71333
Epoch: 277 Batch:   0 Loss: 0.63037 Accuracy: 0.68750
Test Loss: 0.69673 Accuracy: 0.71333
Epoch: 278 Batch:   0 Loss: 0.63037 Accuracy: 0.68750
Test Loss: 0.69672 Accuracy: 0.71333
Epoch: 279 Batch:   0 Loss: 0.63036 Accuracy: 0.68750
Test Loss: 0.69672 Accuracy: 0.71333
Epoch: 280 Batch:   0 Loss: 0.63035 Accuracy: 0.68750
Test Loss: 0.69673 Accuracy: 0.71333
Epoch: 281 Batch:   0 Loss: 0.63032 Accuracy: 0.68750
Test Loss: 0.69673 Accuracy: 0.71333
Epoch: 282 Batch:   0 Loss: 0.63031 Accuracy: 0.68750
Test Loss: 0.69673 Accuracy: 0.71333
Epoch: 283 Batch:   0 Loss: 0.63031 Accuracy: 0.68750

Test Loss: 0.69636 Accuracy: 0.71333
Epoch: 364 Batch:   0 Loss: 0.62913 Accuracy: 0.68750
Test Loss: 0.69634 Accuracy: 0.71333
Epoch: 365 Batch:   0 Loss: 0.62909 Accuracy: 0.68750
Test Loss: 0.69635 Accuracy: 0.71333
Epoch: 366 Batch:   0 Loss: 0.62908 Accuracy: 0.68750
Test Loss: 0.69634 Accuracy: 0.71333
Epoch: 367 Batch:   0 Loss: 0.62907 Accuracy: 0.68750
Test Loss: 0.69633 Accuracy: 0.71333
Epoch: 368 Batch:   0 Loss: 0.62906 Accuracy: 0.68750
Test Loss: 0.69632 Accuracy: 0.71000
Epoch: 369 Batch:   0 Loss: 0.62905 Accuracy: 0.68750
Test Loss: 0.69631 Accuracy: 0.71000
Epoch: 370 Batch:   0 Loss: 0.62901 Accuracy: 0.68750
Test Loss: 0.69630 Accuracy: 0.71000
Epoch: 371 Batch:   0 Loss: 0.62902 Accuracy: 0.68750
Test Loss: 0.69628 Accuracy: 0.71000
Epoch: 372 Batch:   0 Loss: 0.62900 Accuracy: 0.68750
Test Loss: 0.69629 Accuracy: 0.71000
Epoch: 373 Batch:   0 Loss: 0.62898 Accuracy: 0.68750
Test Loss: 0.69628 Accuracy: 0.71000
Epoch: 374 Batch:   0 Loss: 0.62896 Accuracy: 0.68750

...

Test Loss: 0.69357 Accuracy: 0.71333
Epoch: 546 Batch:   0 Loss: 0.62861 Accuracy: 0.68750
Test Loss: 0.69358 Accuracy: 0.71333
Epoch: 547 Batch:   0 Loss: 0.62861 Accuracy: 0.68750
Test Loss: 0.69357 Accuracy: 0.71333
Epoch: 548 Batch:   0 Loss: 0.62858 Accuracy: 0.68750
Test Loss: 0.69357 Accuracy: 0.71333
Epoch: 549 Batch:   0 Loss: 0.62857 Accuracy: 0.68750
Test Loss: 0.69357 Accuracy: 0.71333
Epoch: 550 Batch:   0 Loss: 0.62855 Accuracy: 0.68750
Test Loss: 0.69358 Accuracy: 0.71333
Epoch: 551 Batch:   0 Loss: 0.62856 Accuracy: 0.68750
Test Loss: 0.69358 Accuracy: 0.71333
Epoch: 552 Batch:   0 Loss: 0.62854 Accuracy: 0.68750
Test Loss: 0.69358 Accuracy: 0.71333
Epoch: 553 Batch:   0 Loss: 0.62855 Accuracy: 0.68750
Test Loss: 0.69357 Accuracy: 0.71333
Epoch: 554 Batch:   0 Loss: 0.62852 Accuracy: 0.68750
Test Loss: 0.69358 Accuracy: 0.71333
Epoch: 555 Batch:   0 Loss: 0.62853 Accuracy: 0.68750
Test Loss: 0.69356 Accuracy: 0.71333
Epoch: 556 Batch:   0 Loss: 0.62850 Accuracy: 0.68750

Test Loss: 0.69380 Accuracy: 0.71333
Epoch: 637 Batch:   0 Loss: 0.62833 Accuracy: 0.68750
Test Loss: 0.69382 Accuracy: 0.71333
Epoch: 638 Batch:   0 Loss: 0.62834 Accuracy: 0.68750
Test Loss: 0.69381 Accuracy: 0.71333
Epoch: 639 Batch:   0 Loss: 0.62835 Accuracy: 0.68750
Test Loss: 0.69381 Accuracy: 0.71333
Epoch: 640 Batch:   0 Loss: 0.62835 Accuracy: 0.68750
Test Loss: 0.69384 Accuracy: 0.71333
Epoch: 641 Batch:   0 Loss: 0.62839 Accuracy: 0.68750
Test Loss: 0.69384 Accuracy: 0.71333
Epoch: 642 Batch:   0 Loss: 0.62840 Accuracy: 0.68750
Test Loss: 0.69387 Accuracy: 0.71333
Epoch: 643 Batch:   0 Loss: 0.62839 Accuracy: 0.68750
Test Loss: 0.69387 Accuracy: 0.71333
Epoch: 644 Batch:   0 Loss: 0.62842 Accuracy: 0.68750
Test Loss: 0.69387 Accuracy: 0.71333
Epoch: 645 Batch:   0 Loss: 0.62843 Accuracy: 0.68750
Test Loss: 0.69388 Accuracy: 0.71333
Epoch: 646 Batch:   0 Loss: 0.62845 Accuracy: 0.68750
Test Loss: 0.69389 Accuracy: 0.71333
Epoch: 647 Batch:   0 Loss: 0.62846 Accuracy: 0.68750

Test Loss: 0.69423 Accuracy: 0.71667
Epoch: 728 Batch:   0 Loss: 0.62933 Accuracy: 0.68750
Test Loss: 0.69422 Accuracy: 0.71667
Epoch: 729 Batch:   0 Loss: 0.62932 Accuracy: 0.68750
Test Loss: 0.69422 Accuracy: 0.71667
Epoch: 730 Batch:   0 Loss: 0.62931 Accuracy: 0.68750
Test Loss: 0.69424 Accuracy: 0.71667
Epoch: 731 Batch:   0 Loss: 0.62930 Accuracy: 0.68750
Test Loss: 0.69423 Accuracy: 0.71667
Epoch: 732 Batch:   0 Loss: 0.62928 Accuracy: 0.68750
Test Loss: 0.69422 Accuracy: 0.71667
Epoch: 733 Batch:   0 Loss: 0.62929 Accuracy: 0.68750
Test Loss: 0.69423 Accuracy: 0.71667
Epoch: 734 Batch:   0 Loss: 0.62927 Accuracy: 0.68750
Test Loss: 0.69423 Accuracy: 0.71667
Epoch: 735 Batch:   0 Loss: 0.62928 Accuracy: 0.68750
Test Loss: 0.69423 Accuracy: 0.71667
Epoch: 736 Batch:   0 Loss: 0.62928 Accuracy: 0.68750
Test Loss: 0.69421 Accuracy: 0.71667
Epoch: 737 Batch:   0 Loss: 0.62925 Accuracy: 0.68750
Test Loss: 0.69422 Accuracy: 0.71667
Epoch: 738 Batch:   0 Loss: 0.62925 Accuracy: 0.68750

Test Loss: 0.69425 Accuracy: 0.72000
Epoch: 819 Batch:   0 Loss: 0.62835 Accuracy: 0.68750
Test Loss: 0.69425 Accuracy: 0.72000
Epoch: 820 Batch:   0 Loss: 0.62832 Accuracy: 0.68750
Test Loss: 0.69425 Accuracy: 0.72000
Epoch: 821 Batch:   0 Loss: 0.62831 Accuracy: 0.68750
Test Loss: 0.69426 Accuracy: 0.72000
Epoch: 822 Batch:   0 Loss: 0.62830 Accuracy: 0.68750
Test Loss: 0.69426 Accuracy: 0.72000
Epoch: 823 Batch:   0 Loss: 0.62830 Accuracy: 0.68750
Test Loss: 0.69426 Accuracy: 0.72000
Epoch: 824 Batch:   0 Loss: 0.62830 Accuracy: 0.68750
Test Loss: 0.69426 Accuracy: 0.72000
Epoch: 825 Batch:   0 Loss: 0.62827 Accuracy: 0.68750
Test Loss: 0.69425 Accuracy: 0.72000
Epoch: 826 Batch:   0 Loss: 0.62825 Accuracy: 0.68750
Test Loss: 0.69427 Accuracy: 0.72000
Epoch: 827 Batch:   0 Loss: 0.62826 Accuracy: 0.68750
Test Loss: 0.69427 Accuracy: 0.72000
Epoch: 828 Batch:   0 Loss: 0.62824 Accuracy: 0.68750
Test Loss: 0.69426 Accuracy: 0.72000
Epoch: 829 Batch:   0 Loss: 0.62823 Accuracy: 0.68750

Test Loss: 0.69440 Accuracy: 0.72000
Epoch: 910 Batch:   0 Loss: 0.62754 Accuracy: 0.68750
Test Loss: 0.69439 Accuracy: 0.72000
Epoch: 911 Batch:   0 Loss: 0.62755 Accuracy: 0.68750
Test Loss: 0.69438 Accuracy: 0.72000
Epoch: 912 Batch:   0 Loss: 0.62754 Accuracy: 0.68750
Test Loss: 0.69438 Accuracy: 0.72000
Epoch: 913 Batch:   0 Loss: 0.62752 Accuracy: 0.68750
Test Loss: 0.69435 Accuracy: 0.72000
Epoch: 914 Batch:   0 Loss: 0.62746 Accuracy: 0.68750
Test Loss: 0.69439 Accuracy: 0.72000
Epoch: 915 Batch:   0 Loss: 0.62749 Accuracy: 0.68750
Test Loss: 0.69442 Accuracy: 0.72000
Epoch: 916 Batch:   0 Loss: 0.62750 Accuracy: 0.68750
Test Loss: 0.69441 Accuracy: 0.72000
Epoch: 917 Batch:   0 Loss: 0.62749 Accuracy: 0.68750
Test Loss: 0.69442 Accuracy: 0.72000
Epoch: 918 Batch:   0 Loss: 0.62750 Accuracy: 0.68750
Test Loss: 0.69441 Accuracy: 0.72000
Epoch: 919 Batch:   0 Loss: 0.62749 Accuracy: 0.68750
Test Loss: 0.69442 Accuracy: 0.72000
Epoch: 920 Batch:   0 Loss: 0.62748 Accuracy: 0.68750

# Reload Model

In [11]:
train = np.load('earable_dataset' + '/train.npy')
test = np.load('earable_dataset' + '/test.npy')
x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]
# Convert y to one-hot
minval = min(min(y_train), min(y_test))
numClasses = max(y_train) - min(y_train) + 1
numClasses = max(numClasses, max(y_test) - min(y_test) + 1)
numClasses = int(numClasses)
y_train = helpermethods.to_onehot(y_train, numClasses, minlabel=minval)
y_test = helpermethods.to_onehot(y_test, numClasses, minlabel=minval)
dataDimension = x_train.shape[1]

W = np.load(model_dir + '/W.npy')
B = np.load(model_dir + '/B.npy')
Z = np.load(model_dir + '/Z.npy')
gamma = np.load(model_dir + '/gamma.npy')

n_dim = inputDimension = W.shape[0]
projectionDimension = W.shape[1]
numPrototypes = B.shape[1]
numOutputLabels = Z.shape[0]

dense = ProtoNNLayer( inputDimension, projectionDimension, numPrototypes, numOutputLabels, gamma )

model = keras.Sequential([
    keras.layers.InputLayer(n_dim),
    dense
])

dummy_tensor = tf.convert_to_tensor( np.zeros((1,n_dim), np.float32) )
out_tensor = model( dummy_tensor )

dense.set_weights( [W, B, Z] )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
proto_nn_layer (ProtoNNLayer (None, 9)                 6230      
Total params: 6,230
Trainable params: 6,230
Non-trainable params: 0
_________________________________________________________________
